# Лабораторная работа 6: Логистическая регрессия

## Реализация без использования библиотек машинного обучения, только NumPy и Pandas

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    confusion_matrix,
)
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler


# Настройка отображения графиков
%matplotlib inline
plt.rcParams["figure.figsize"] = (10, 6)
sns.set_style("whitegrid")

## 1. Загрузка и предварительная обработка данных

In [ ]:
# Загрузка данных
df = pd.read_csv("data/Titanic.csv")

print("1. ПРЕДВАРИТЕЛЬНАЯ ОБРАБОТКА ДАННЫХ")
print("=" * 50)

# Проверка на отсутствующие значения
print("Проверка отсутствующих значений:")
print(df.isnull().sum())
print()

# Информация о датасете
print("Информация о датасете:")
print(df.info())
print()

# Выделение признаков и целевой переменной
X = df.drop("Survived", axis=1)
y = df["Survived"]

print(f"Размерность признаков: {X.shape}")
print(f"Размерность целевой переменной: {y.shape}")
print()

## 2. Статистика и визуализация данных

In [ ]:
print("2. СТАТИСТИКА И ВИЗУАЛИЗАЦИЯ ДАННЫХ")
print("=" * 50)

# Базовая статистика
print("Статистика датасета:")
print(df.describe())
print()

In [ ]:
# Визуализация распределения классов
plt.figure(figsize=(10, 6))
sns.countplot(x="Survived", data=df)
plt.title("Распределение выживших пассажиров")
plt.xlabel("Выжил (1 - Да, 0 - Нет)")
plt.ylabel("Количество")
plt.show()

### 2.1. Визуализация распределения признаков

In [ ]:
print("2.1. ВИЗУАЛИЗАЦИЯ РАСПРЕДЕЛЕНИЯ ПРИЗНАКОВ")
print("=" * 50)

# Создание сетки графиков
fig, axes = plt.subplots(2, 3, figsize=(15, 10))
fig.suptitle("Распределение признаков датасета Титаника", fontsize=16, fontweight="bold")

# Построение гистограмм для каждого признака
features = ["Pclass", "Age", "SibSp", "Parch", "Fare"]
for i, feature in enumerate(features):
    row = i // 3
    col = i % 3

    axes[row, col].hist(df[feature], bins=15, color="skyblue", edgecolor="black", alpha=0.7)
    axes[row, col].set_title(f"Распределение {feature}", fontweight="bold")
    axes[row, col].set_xlabel(feature)
    axes[row, col].set_ylabel("Частота")
    axes[row, col].grid(True, alpha=0.3)

    mean_val = df[feature].mean()
    axes[row, col].axvline(
        mean_val,
        color="red",
        linestyle="--",
        linewidth=2,
        label=f"Среднее: {mean_val:.2f}",
    )
    axes[row, col].legend()

plt.tight_layout()
plt.show()

## 3. Разделение данных и предобработка

In [ ]:
print("3. РАЗДЕЛЕНИЕ ДАННЫХ И ПРЕДОБРАБОТКА")
print("=" * 50)


# Обработка категориальных признаков
def preprocess_data(df):
    df_processed = df.copy()

    # Кодируем пол (Sex)
    df_processed["Sex"] = df_processed["Sex"].map({"male": 0, "female": 1})

    # Кодируем порт посадки (Embarked)
    embarked_mapping = {"S": 0, "C": 1, "Q": 2}
    df_processed["Embarked"] = df_processed["Embarked"].map(embarked_mapping)

    # Заполняем пропущенные значения
    df_processed["Age"] = df_processed["Age"].fillna(df_processed["Age"].median())
    df_processed["Embarked"] = df_processed["Embarked"].fillna(0)
    df_processed["Fare"] = df_processed["Fare"].fillna(df_processed["Fare"].median())

    # Удаляем столбцы, которые сложно кодировать
    columns_to_drop = ["Name", "Ticket", "Cabin", "PassengerId"]
    df_processed = df_processed.drop(
        columns=[col for col in columns_to_drop if col in df_processed.columns]
    )

    return df_processed


# Предобработка данных
X_processed = preprocess_data(X)

print("Признаки после предобработки:")
print(X_processed.columns.tolist())
print()

# Масштабирование признаков (стандартизация)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_processed)

# Разделение на обучающую и тестовую выборки
X_train_scaled, X_test_scaled, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2)

print(f"Обучающая выборка: {X_train_scaled.shape}")
print(f"Тестовая выборка: {X_test_scaled.shape}")
print(f"Доля выживших в обучающей выборке: {y_train.mean():.3f}")
print()

## 4. Реализация логистической регрессии

In [ ]:
class LogisticRegression:
    def __init__(self):
        self.weights = None
        self.bias = None
        self.losses = []

    def sigmoid(self, t):
        # Добавляем ограничение для численной стабильности
        t = np.clip(t, -500, 500)  # Предотвращаем переполнение
        return 1 / (1 + np.exp(-t))

    def log_loss(self, y_actual, y_predicted):
        # Добавляем эпсилон для избежания log(0)
        epsilon = 1e-15
        y_predicted = np.clip(y_predicted, epsilon, 1 - epsilon)
        return -np.mean(y_actual * np.log(y_predicted) + (1 - y_actual) * np.log(1 - y_predicted))

    def gradient_descent(self, X_train, y_train, iterations=1000, learning_rate=0.01):
        objects_num, characteristics_num = X_train.shape
        self.weights = np.random.normal(0, 0.01, characteristics_num)
        self.bias = 0
        self.losses = []

        for iteration in range(1, iterations + 1):
            t = np.dot(X_train, self.weights) + self.bias
            z = self.sigmoid(t)

            # Можно вычислить dz/dw и dLoss/dz -> dLoss/dw = (1/n) * X^T * (Z - Y)
            dw = (1 / objects_num) * np.dot(X_train.T, (z - y_train))
            # Аналогично вычисляется dLoss/db
            db = (1 / objects_num) * np.sum(z - y_train)

            # w = w - α * ∇J(w)
            self.weights -= learning_rate * dw
            self.bias -= learning_rate * db

            if iteration % 100 == 0:
                loss = self.log_loss(y_train, z)
                self.losses.append(loss)

    def newton_optimization(self, X_train, y_train, iterations=1000):
        objects_num, characteristics_num = X_train.shape
        self.weights = np.random.normal(0, 0.01, characteristics_num)
        self.bias = 0
        self.losses = []

        for iteration in range(1, iterations + 1):
            t = np.dot(X_train, self.weights) + self.bias
            z = self.sigmoid(t)

            dw = (1 / objects_num) * np.dot(X_train.T, (z - y_train))
            db = (1 / objects_num) * np.sum(z - y_train)

            W = np.diag(z * (1 - z))
            hessian = (1 / objects_num) * (X_train.T @ W @ X_train)

            hessian += 1e-6 * np.eye(hessian.shape[0])

            try:
                self.weights -= np.linalg.solve(hessian, dw)
                self.bias -= db
            except np.linalg.LinAlgError:
                self.weights -= np.linalg.pinv(hessian) @ dw
                self.bias -= db

            if iteration % 100 == 0:
                loss = self.log_loss(y_train, z)
                self.losses.append(loss)

    def predict(self, X_test, threshold=0.5):
        t = np.dot(X_test, self.weights) + self.bias
        z = self.sigmoid(t)
        return (z > threshold).astype(int)

## 5. Исследование гиперпараметров

In [ ]:
print("5. ИССЛЕДОВАНИЕ ГИПЕРПАРАМЕТРОВ")
print("=" * 50)


# Функция для вычисления метрик
def calculate_metrics(y_true, y_pred):
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, zero_division=0)
    recall = recall_score(y_true, y_pred, zero_division=0)
    f1 = f1_score(y_true, y_pred, zero_division=0)

    return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1_score": f1}


# Параметры для исследования
learning_rates = [0.001, 0.01, 0.1]  # Измененные значения
iterations_list = [100, 500, 1000, 2000]  # Добавлены больше итераций

print("МОДЕЛЬ 1: ГРАДИЕНТНЫЙ СПУСК")
print("-" * 40)

results_gd = []

for lr in learning_rates:
    for iters in iterations_list:
        model = LogisticRegression()
        model.gradient_descent(
            X_train_scaled,
            y_train.values,
            iterations=iters,
            learning_rate=lr,
        )

        y_pred = model.predict(X_test_scaled)

        # Проверим распределение предсказаний
        unique, counts = np.unique(y_pred, return_counts=True)
        pred_dist = dict(zip(unique, counts))

        metrics = calculate_metrics(y_test.values, y_pred)

        results_gd.append(
            {
                "method": "Градиентный спуск",
                "learning_rate": lr,
                "iterations": iters,
                "pred_distribution": pred_dist,
                **metrics,
            }
        )

        print(
            f"LR: {lr}, Iter: {iters}, Pred: {pred_dist}, "
            f"Accuracy: {metrics['accuracy']:.4f}, F1: {metrics['f1_score']:.4f}"
        )

print("\nМОДЕЛЬ 2: ОПТИМИЗАЦИЯ НЬЮТОНА")
print("-" * 40)

results_nt = []

for iters in iterations_list:
    model = LogisticRegression()
    model.newton_optimization(X_train_scaled, y_train.values, iterations=iters)

    y_pred = model.predict(X_test_scaled)

    unique, counts = np.unique(y_pred, return_counts=True)
    pred_dist = dict(zip(unique, counts))

    metrics = calculate_metrics(y_test.values, y_pred)

    results_nt.append(
        {
            "method": "Метод Ньютона",
            "learning_rate": "-",
            "iterations": iters,
            "pred_distribution": pred_dist,
            **metrics,
        }
    )

    print(
        f"Iter: {iters}, Pred: {pred_dist}, "
        f"Accuracy: {metrics['accuracy']:.4f}, F1: {metrics['f1_score']:.4f}"
    )

## 6. Лучшая модель и итоговые результаты

In [ ]:
print("6. ЛУЧШАЯ МОДЕЛЬ И ИТОГОВЫЕ РЕЗУЛЬТАТЫ")
print("=" * 50)

# Находим лучшую модель по F1-score
all_results = pd.DataFrame(results_gd + results_nt)
best_result = all_results.loc[all_results["f1_score"].idxmax()]

print("ЛУЧШАЯ КОНФИГУРАЦИЯ:")
print(f"Метод: {best_result['method']}")
print(f"Learning rate: {best_result['learning_rate']}")
print(f"Количество итераций: {best_result['iterations']}")
print(f"Распределение предсказаний: {best_result['pred_distribution']}")
print()
print("МЕТРИКИ ЛУЧШЕЙ МОДЕЛИ:")
print(f"Accuracy: {best_result['accuracy']:.4f}")
print(f"Precision: {best_result['precision']:.4f}")
print(f"Recall: {best_result['recall']:.4f}")
print(f"F1-Score: {best_result['f1_score']:.4f}")
print()

# Обучение лучшей модели
if best_result["method"] == "Градиентный спуск":
    best_model = LogisticRegression()
    best_model.gradient_descent(
        X_train_scaled,
        y_train.values,
        iterations=int(best_result["iterations"]),
        learning_rate=best_result["learning_rate"],
    )
else:
    best_model = LogisticRegression()
    best_model.newton_optimization(
        X_train_scaled.values, y_train.values, iterations=int(best_result["iterations"])
    )

# Предсказания лучшей модели
y_pred_best = best_model.predict(X_test_scaled)

# Матрица ошибок

cm = confusion_matrix(y_test.values, y_pred_best)

plt.figure(figsize=(6, 5))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues")
plt.title("Матрица ошибок лучшей модели")
plt.ylabel("Истинные значения")
plt.xlabel("Предсказанные значения")
plt.show()